In [ ]:
import os, sys, shutil
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import random
from datetime import datetime
from torchvision import transforms
import torch
from scipy.stats import binom_test
# os.chdir("..")
experiment_version = 4
os.makedirs(f"stimuli_v{experiment_version}", exist_ok=True)
os.makedirs(f"responses_v{experiment_version}", exist_ok=True)
os.makedirs(f"dataframes_v{experiment_version}", exist_ok=True)

# CREATE EXPERIMENT DATAFRAME AND TRIAL FILES FOR MEADOWS

In [ ]:
#Experiment column key:
# 1: Experiment 1, mindeye vs second sight
# 2: Experiment 2, second sight two way identification
# 3: Experiment 3, mental imagery two way identification
df_exp = pd.DataFrame(columns=["experiment", "stim1", "stim2", "stim3", "sample", "subject", "target_on_left", "catch_trial", "rep"])
i=0
random_count = 0
gt_tensor_block = torch.load("raw_stimuli/all_images_425.pt")
for subj in [1,2,5,7]: #1,2,5,7
    subject_enhanced_recons_40 = torch.load(f"raw_stimuli/final_subj0{subj}_pretrained_40sess_24bs_all_enhancedrecons.pt")
    subject_unclip_recons_40 = torch.load(f"raw_stimuli/final_subj0{subj}_pretrained_40sess_24bs_all_recons.pt")
    subject_enhanced_recons_1 = torch.load(f"raw_stimuli/final_subj0{subj}_pretrained_1sess_24bs_all_enhancedrecons.pt")
    subject_braindiffuser_recons_1 = torch.load(f"raw_stimuli/subj0{subj}_brain_diffuser_750_all_recons.pt")
    #Experiment 1, mindeye two way identification
    random_indices = random.sample(range(1000), 300)
    for sample in tqdm(random_indices):
        
        # Get random sample to compare against
        random_number = random.choice([x for x in range(1000) if x != sample])
        # Extract the stimulus images from tensor blocks and save as pngs to stimuli folder
        gt_sample = transforms.ToPILImage()(gt_tensor_block[sample])
        sample_enhanced_recons_40 = transforms.ToPILImage()(subject_enhanced_recons_40[sample]).resize((425,425))
        random_enhanced_recons_40 = transforms.ToPILImage()(subject_enhanced_recons_40[random_number]).resize((425,425))
        sample_enhanced_recons_40.save(f"stimuli_v{experiment_version}/{sample}_subject{subj}_mindeye_enhanced_40.png")
        random_enhanced_recons_40.save(f"stimuli_v{experiment_version}/{random_number}_subject{subj}_mindeye_enhanced_40.png")
        gt_sample.save(f"stimuli_v{experiment_version}/{sample}_ground_truth.png")
        
        # Configure stimuli names and order in experiment dataframe
        sample_names = [f"{random_number}_subject{subj}_mindeye_enhanced_40", f"{sample}_subject{subj}_mindeye_enhanced_40"]
        order = random.randrange(2)
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        gt_sample = f"{sample}_ground_truth"
        df_exp.loc[i] = {"experiment" : 1, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                         "target_on_left" : order == 1, "catch_trial" : None, "rep" : 0}
        i+=1
        
    #Experiment 2, refined vs unrefined
    random_indices = random.sample(range(1000), 300)
    for sample in tqdm(random_indices):
        
        # Extract the stimulus images from tensor blocks and save as pngs to stimuli folder
        gt_sample = transforms.ToPILImage()(gt_tensor_block[sample])
        sample_enhanced_recons_40 = transforms.ToPILImage()(subject_enhanced_recons_40[sample]).resize((425,425))
        sample_unclip_recons_40 = transforms.ToPILImage()(subject_unclip_recons_40[sample]).resize((425,425))
        sample_enhanced_recons_40.save(f"stimuli_v{experiment_version}/{sample}_subject{subj}_mindeye_enhanced_40.png")
        sample_unclip_recons_40.save(f"stimuli_v{experiment_version}/{sample}_subject{subj}_mindeye_unclip_40.png")
        gt_sample.save(f"stimuli_v{experiment_version}/{sample}_ground_truth.png")
        
        # Configure stimuli names and order in experiment dataframe
        sample_names = [f"{sample}_subject{subj}_mindeye_unclip_40", f"{sample}_subject{subj}_mindeye_enhanced_40"]
        order = random.randrange(2)
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        gt_sample = f"{sample}_ground_truth"
        df_exp.loc[i] = {"experiment" : 2, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                         "target_on_left" : order == 1, "catch_trial" : None, "rep" : 0}
        i+=1
        
    #Experiment 3, refined 1 session vs brain diffuser 1 session
    random_indices = random.sample(range(1000), 300)
    for sample in tqdm(random_indices):
        
        # Extract the stimulus images from tensor blocks and save as pngs to stimuli folder
        gt_sample = transforms.ToPILImage()(gt_tensor_block[sample])
        sample_enhanced_recons_1 = transforms.ToPILImage()(subject_enhanced_recons_1[sample]).resize((425,425))
        sample_braindiffuser_1 = transforms.ToPILImage()(subject_braindiffuser_recons_1[sample]).resize((425,425))
        sample_enhanced_recons_1.save(f"stimuli_v{experiment_version}/{sample}_subject{subj}_mindeye_enhanced_1.png")
        sample_braindiffuser_1.save(f"stimuli_v{experiment_version}/{sample}_subject{subj}_braindiffuser_1.png")
        gt_sample.save(f"stimuli_v{experiment_version}/{sample}_ground_truth.png")
        
        # Configure stimuli names and order in experiment dataframe
        sample_names = [f"{sample}_subject{subj}_braindiffuser_1", f"{sample}_subject{subj}_mindeye_enhanced_1"]
        order = random.randrange(2)
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        gt_sample = f"{sample}_ground_truth"
        df_exp.loc[i] = {"experiment" : 3, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                         "target_on_left" : order == 1, "catch_trial" : None, "rep" : 0}
        i+=1
df_exp = df_exp.sample(frac=1)
print(len(df_exp))
print(df_exp)

In [ ]:
# Check if all images are present in final stimuli folder
count_not_found = 0
stim_path = f"stimuli_v{experiment_version}/"
for index, row in df_exp.iterrows():
    if not (os.path.exists(f"{stim_path}{row['stim1']}.png")):
        print(f"{row['stim1']}.png")
        count_not_found += 1
    if not (os.path.exists(f"{stim_path}{row['stim2']}.png")):
        print(f"{row['stim2']}.png")
        count_not_found += 1
    if not (os.path.exists(f"{stim_path}{row['stim3']}.png")):
        print(f"{row['stim3']}.png")
        count_not_found += 1
print(count_not_found)

In [ ]:
#Add participant ID column
pIDs = []
for i in range(len(df_exp)):
    pIDs.append(i // 60)
df_exp.insert(0, "pID", pIDs)
print(len(df_exp[(df_exp['pID'] == 0)]))
#Add catch trials within each pID section
for pID in range(max(pIDs)):
    df_pid = df_exp[(df_exp['experiment'] == 1) & (df_exp['pID'] == pID)]
    
    # Ground truth catch trials
    gt_catch_trials = df_pid.sample(n=9)
    gt_catch_trials['catch_trial'] = "ground_truth"
    for index, row in gt_catch_trials.iterrows():
        
        order = random.randrange(2)
        ground_truth = row['stim1']
        stims = [row['stim2'], ground_truth]
        
        gt_catch_trials.at[index, 'stim2'] = stims.pop(order)
        gt_catch_trials.at[index, 'stim3'] = stims.pop()
        # Target on left here means the ground truth repeat is on the left
        gt_catch_trials.at[index, 'target_on_left'] = (order == 1)
        
    # repeated trial catch trials, first sample indices
    sampled_indices = df_pid.sample(n=9).index
    #mark the trials at these indices as catch trials
    df_exp.loc[sampled_indices]['catch_trial'] = "repeat"
    #create duplicate trials for these samples to repeat
    repeat_catch_trials_rep1 = df_exp.loc[sampled_indices].copy()
    repeat_catch_trials_rep2 = df_exp.loc[sampled_indices].copy()
    repeat_catch_trials_rep1['rep'] = 1
    repeat_catch_trials_rep2['rep'] = 2
    
    
    df_exp = pd.concat([df_exp, gt_catch_trials, repeat_catch_trials_rep1, repeat_catch_trials_rep2])
    
df_exp = df_exp.sample(frac=1).sort_values(by='pID', kind='mergesort')
print(len(df_exp))
print(len(df_exp[(df_exp['pID'] == 0)]))

In [ ]:

df_exp.to_csv(f'dataframes_v{experiment_version}/experiment_v{experiment_version}.csv', index=False)

df_exp_tsv = df_exp[['pID', 'stim1', 'stim2', 'stim3']].copy()
df_exp_tsv.to_csv(f"dataframes_v{experiment_version}/meadow_trials_v{experiment_version}.tsv", sep="\t", index=False, header=False) 

# THE FOLLOWING CELLS ARE FOR PROCESSING RESPONSES

In [ ]:
response_path = f"responses_v{experiment_version}/"
dataframe_path = f"dataframes_v{experiment_version}/"
df_experiment = pd.read_csv(dataframe_path + f"experiment_v{experiment_version}.csv")
response_version = "2"
df_responses = pd.read_csv(f"{response_path}deployment_v{response_version}.csv")
print(df_responses)

In [ ]:
df_responses.head()
df_trial = pd.DataFrame(columns=["experiment", "stim1", "stim2", "stim3", "sample", "subject", "target_on_left", "method", "catch_trial", "rep", "picked_left", "participant"])
df_experiment['picked_left'] = None
for index, row in tqdm(df_responses.iterrows()):
    if row['label'] == row['stim2_id']:
        picked_left = True
    elif row['label'] == row['stim3_id']:
        picked_left = False
    else:
        print("Error")
        break
    start_timestamp = row['time_trial_start']
    end_timestamp = row['time_trial_response']
    start = datetime.fromisoformat(start_timestamp.replace("Z", "+00:00"))
    end = datetime.fromisoformat(end_timestamp.replace("Z", "+00:00"))
    # Calculate the difference in seconds
    time_difference_seconds = (end - start).total_seconds()
    
    df_trial.loc[index] = df_experiment[(df_experiment['stim1'] == row['stim1_name']) & (df_experiment['stim2'] == row['stim2_name']) & (df_experiment['stim3'] == row['stim3_name'])].iloc[0]
    df_trial.loc[index, 'picked_left'] = picked_left
    df_trial.loc[index, 'participant'] = row['participation']
    df_trial.loc[index, 'response_time'] = time_difference_seconds
    
df_trial["picked_target"] = df_trial["picked_left"] == df_trial["target_on_left"]
print(df_trial)

In [ ]:
# number of participants
print("Total participants:", len(df_trial["participant"].unique()))

gt_failures = df_trial[(df_trial['catch_trial'] == 'ground_truth') & (df_trial['picked_target'] == False)].groupby('participant').size()
# Identify participants who failed more than 1 ground truth catch trial
participants_to_remove_rule1 = gt_failures[gt_failures > 1].index.tolist()
print("Participants to remove 1:", participants_to_remove_rule1)

# Remove participants who failed the repeat catch trial, and gave different responses for identical trials
repeat_trials = df_trial[df_trial['rep'] > 0]
grouped_repeat_trials = repeat_trials.groupby(['stim1', 'stim2', 'stim3'])
participant_failures = {}
# Iterate through groups to check consistency in "picked_target" across repetitions
for _, group in grouped_repeat_trials:
    if group['picked_target'].nunique() != 1:  # Inconsistent "picked_target" within the group
        for participant in group['participant'].unique(): 
            participant_failures[participant] = participant_failures.get(participant, 0) + 1

# Identify participants who failed at least one set of trial repetitions
participants_to_remove_rule2 = [participant for participant, failures in participant_failures.items() if failures > 1]
print("Participants to remove 2:", participants_to_remove_rule2)

participants_to_remove = set(participants_to_remove_rule1).union(set(participants_to_remove_rule2))
filtered_df = df_trial[~df_trial['participant'].isin(participants_to_remove)]
print("Clean participants:", len(filtered_df["participant"].unique()))
print(len(df_trial), len(filtered_df))
print(participants_to_remove)
filtered_df.to_csv(f'{dataframe_path}filtered_responses_v{response_version}.csv', index=False)

In [ ]:
# Load filtered responses
filtered_df = pd.read_csv(f'{dataframe_path}filtered_responses_v{response_version}.csv')
# Filter out catch trials
df_trial_exp = filtered_df[(filtered_df['catch_trial'].isnull() & (filtered_df['rep'] == 0))]

# Grab results from an individual experiment and print them out
df_trial_exp1 = df_trial_exp[df_trial_exp['experiment'] == 3]

# Perform a binomial test
# The null hypothesis is that the probability of success is 0.5 (chance level)
p_value = binom_test(df_trial_exp1['picked_target'].sum(), n=len(df_trial_exp1['picked_target']), p=0.5, alternative='two-sided')

print("Number of experiment trials:", len(df_trial_exp1))
print("Success rate: ", len(df_trial_exp1[df_trial_exp1["picked_target"]]) / len(df_trial_exp1))
print(f'P-value: {p_value}')

In [ ]:
import shutil
from PIL import Image, ImageDraw, ImageFont

# Filter for experiment 3 rows where picked_target is false
df_exp3_failures = df_trial_exp[df_trial_exp['experiment'] == 3]
df_exp3_failures = df_exp3_failures[df_exp3_failures['picked_target'] == False]

# Create the "brain_diffuser_failures" folder if it doesn't exist
os.makedirs("brain_diffuser_failures_tiled", exist_ok=True)

# Copy the stimuli from stimuli_v4 to the "brain_diffuser_failures" folder
# Set the dimensions for the concatenated image
width = 3 * 425
height = 450

# Create a blank canvas for the concatenated image
concatenated_image = Image.new('RGB', (width, height), (255, 255, 255))
draw = ImageDraw.Draw(concatenated_image)

# Set the font properties for the title captions
font = ImageFont.truetype("arial.ttf", 16)

# Iterate over the rows in df_exp3_failures
for index, row in df_exp3_failures.iterrows():
    # Get the paths for the stimuli images
    stim1_path = f"stimuli_v4/{row['stim1']}.png"
    stim2_path = f"stimuli_v4/{row['stim2']}.png"
    stim3_path = f"stimuli_v4/{row['stim3']}.png"
    
    # Open the stimuli images
    stim1_image = Image.open(stim1_path)
    stim2_image = Image.open(stim2_path)
    stim3_image = Image.open(stim3_path)
    
    # Resize the stimuli images to match the desired dimensions
    stim1_image = stim1_image.resize((425, 425))
    stim2_image = stim2_image.resize((425, 425))
    stim3_image = stim3_image.resize((425, 425))
    
    # Calculate the positions for the stimuli images
    x1 = 0
    x2 = 425
    x3 = 2 * 425
    y = 0
    
    # Paste the stimuli images onto the concatenated image
    concatenated_image.paste(stim1_image, (x1, y))
    concatenated_image.paste(stim2_image, (x2, y))
    concatenated_image.paste(stim3_image, (x3, y))
    
    # Add the title captions for each image
    draw.text((x1, y + 425), f"Stim1 (GT): {row['stim1']}", font=font, fill=(0, 0, 0))
    draw.text((x2, y + 425), f"Stim2: {row['stim2']}", font=font, fill=(0, 0, 0))
    draw.text((x3, y + 425), f"Stim3: {row['stim3']}", font=font, fill=(0, 0, 0))

    # Save the concatenated image
    concatenated_image.save(f"brain_diffuser_failures_tiled/{index}.png")
